In [47]:
from pathlib import Path

import numpy as np
import pandas as pd

In [48]:
df = pd.read_csv("/Users/sanathnair/Developer/trading/reports/original/report_01_03.csv")
df

In [97]:
def modify(ibkr_file):
    df = pd.read_csv(ibkr_file)
    df["Broker Executed"] = pd.to_datetime(
        df["Unnamed: 3"], format = "%I:%M:%S %p"
    ) - pd.Timedelta(hours = 3)
    df = df.drop(
        columns = [
            "Acct ID",
            "Trade Date/Time",
            "Proceeds",
            "Unnamed: 3",
        ]
    )
    df["Quantity"] = pd.to_numeric(df["Quantity"])
    df["Quantity"] = df["Quantity"].abs()
    df["Dollar Amt"] = np.round(df["Quantity"] * df["Price"], 4)
    df = df.rename(columns = {"Type": "Action", "Quantity": "Size"})
    df["Split"] = False
    df["Expiration"] = np.nan
    df["Strike"] = np.nan
    df["Option Type"] = np.nan
    df.loc[df["Action"] == "BUY", "Action"] = "Buy"
    df.loc[df["Action"] == "SELL", "Action"] = "Sell"
    new_file = ibkr_file.parent / (ibkr_file.stem + "_new.csv")
    df.to_csv(new_file, index = False)

In [88]:
p = Path("/Users/sanathnair/Developer/trading/data/ibkr/ibkr_01_03.csv")


In [50]:
ib = get_ibkr_report(Path("/Users/sanathnair/Developer/trading/data/ibkr/ibkr_01_03.csv"))
ib

In [51]:
df_if_rows = df[df["Broker"] == "IF"]
df_if_rows

In [62]:
df_if_rows = df[df["Broker"] == "IF"]
res = pd.merge(df_if_rows, ib, on = ["Symbol", "Action"], how = "outer", suffixes = (None, "_y"))
res["Broker Executed"] = res["Broker Executed_y"]
res["Price"] = res["Price_y"]
res["Dollar Amt"] = res["Dollar Amt_y"]
res["Size"] = res["Size_y"]
res = res.drop(columns = ['Size_y', 'Price_y', 'Broker Executed_y', 'Dollar Amt_y',
                          'Split_y', 'Expiration', 'Strike', 'Option Type'])
res = res[res["Broker Executed"].notna()]
df = df.drop(df_if_rows.index)
df = pd.concat([df, res], axis = 0, ignore_index = True)

In [64]:
df

In [82]:
df = pd.read_csv("/Users/sanathnair/Developer/trading/reports/original/report_02_06.csv")
df

In [68]:
def get_schwab_report(schwab_file: Path) -> pd.DataFrame:
    df = pd.read_csv(schwab_file)
    df["Date"] = pd.to_datetime(df["Date"])
    if "Expiration" not in df.columns:
        df["Expiration"] = np.nan
        df["Strike"] = np.nan
        df["Option Type"] = np.nan
    else:
        df["Expiration"] = pd.to_datetime(df["Expiration"])
    return df

In [83]:
sb = get_schwab_report(Path("/Users/sanathnair/Developer/trading/data/schwab/schwab_02_06.csv"))
sb

In [84]:
df_sb = df[df["Broker"] == "SB"]
df_sb

In [81]:
res = pd.merge(df_sb, sb, on = ["Symbol", "Action"], how = "outer", suffixes = (None, "_y"))
res["Price"] = res["Price_y"]
res["Dollar Amt"] = res["Dollar Amt_y"]
res["Size"] = res["Size_y"]
res["Broker"] = "SB"
res = res.drop(
    columns = ['Size_y', 'Price_y', 'Dollar Amt_y', 'Expiration', 'Strike',
               'Option Type'])
df = df.drop(df_sb.index)
df = pd.concat([df, res], axis = 0, ignore_index = True)
df